In [ ]:
import os
from ctc_src import  *
os.chdir("/home/hemant/E2E_NER-Through-Speech/S2T/")
from opts import add_decoder_args, add_inference_args
from utils import load_model

import numpy as np
import torch
from tqdm.auto import tqdm
from data.data_loader import SpectrogramParser

from opts import add_decoder_args, add_inference_args
from utils import load_model

In [4]:
labels = "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ "
alpha, beta, lm, oov_weight =1.99, 2, '/home/hemant/4_gram.arpa', 1
scorer = Scorer(alpha,beta,lm,oov_weight)

In [6]:
def read_csv(d):
    with open(d,"r") as f:
        return f.readlines()
    
torch.set_grad_enabled(False)
device = torch.device("cuda")
model = load_model(device, "/home/hemant/asr_wm/models/deep/final.pth")
spect_parser = SpectrogramParser(model.audio_conf, normalize=True)

torch.cuda.set_device(int(0))

In [ ]:
total_cer, total_wer, num_tokens, num_chars = 0, 0, 0, 0
output = []

for i in tqdm(csv[:1]):
    audio_path, reference_path = i.split(",")

    spect = spect_parser.parse_audio(audio_path).contiguous()
    spect = spect.view(1, 1, spect.size(0), spect.size(1))
    spect = spect.to(device)

    input_sizes = torch.IntTensor([spect.size(3)]).int()
    out, output_sizes = model(spect, input_sizes)
    out = out.cpu().detach().numpy()[0]

    transcript = ctc_beam_search(out,labels,prune,beam_width,lm,alpha,beta)
#     transcript = ctc_best_path(out,labels)
    with open(reference_path.replace("\n",""),"r") as f:
        reference = f.readline()
        
#     break
    output.append([transcript,reference])
    wer_inst = wer_(transcript,reference)
    cer_inst = cer_(transcript, reference)
    total_wer += wer_inst
    total_cer += cer_inst
    num_tokens += len(reference.split(' '))
    num_chars += len(reference.replace(' ', ''))

        
wer = (float(total_wer) / num_tokens)*100
cer = (float(total_cer) / num_chars)*100
print('Test Summary \t'
    'Average WER {wer:.3f}\t'
    'Average CER {cer:.3f}\t?'.format(wer=wer, cer=cer))

In [ ]:
alpha, beta = np.linspace(0.2,2,15), np.linspace(2,10,5)

In [ ]:
alpha

In [ ]:
beta

In [ ]:
alpha, beta = np.linspace(0.2,2,15), np.linspace(0,6,15)
values = [[i,j] for j in beta for i in alpha]

total_cer, total_wer, num_tokens, num_chars = 0, 0, 0, 0
lm = lm_w
output = []

for alpha,beta in tqdm(values):
    for i in csv:
        audio_path, reference_path = i.split(",")

        spect = spect_parser.parse_audio(audio_path).contiguous()
        spect = spect.view(1, 1, spect.size(0), spect.size(1))
        spect = spect.to(device)

        input_sizes = torch.IntTensor([spect.size(3)]).int()
        out, output_sizes = model(spect, input_sizes)
        out = out.cpu().detach().numpy()[0]

        if decoding == "greedy": transcript = ctc_best_path(out,labels)
        elif decoding == "beam_w": transcript = ctc_beam_search(out,labels,prune,beam_width,lm,alpha,beta)
        elif decoding == "beam_c": transcript = ctc_beam_search_clm(out,labels,prune,beam_width,lm,alpha=alpha,beta=beta)

        with open(reference_path.replace("\n",""),"r") as f:
            reference = f.readline()
        wer_inst = wer_(transcript,reference)
        cer_inst = cer_(transcript, reference)
        total_wer += wer_inst
        total_cer += cer_inst
        num_tokens += len(reference.split(' '))
        num_chars += len(reference.replace(' ', ''))
    np.save("/home/hemant/ctc_decoders/abw.npy",np.array(output))
    wer = (float(total_wer) / num_tokens)*100
    cer = (float(total_cer) / num_chars)*100
    output.append([alpha,beta,wer])
    print("aplha: ",alpha,"beta: ",beta)
    print('Test Summary \t'
        'Average WER {wer:.3f}\t'
        'Average CER {cer:.3f}\t'.format(wer=wer, cer=cer))